# Pre trained network
![imagen](https://www.researchgate.net/publication/336874848/figure/fig1/AS:819325225144320@1572353764073/Illustrations-of-transfer-learning-a-neural-network-is-pretrained-on-ImageNet-and.png)

Estas son las arquitecturas de redes neuronales más utilizadas en la comunidad. Para más detalle sobre el funcionamiento de cada red, consultar el [Hands on Machine Learning for Python](https://learning.oreilly.com/library/view/hands-on-machine-learning/9781492032632/ch14.html#cnn_chapter).
* VGG-16
* VGG-19
* Inception V3
* XCeption
* ResNet-50

Las redes se pueden incorporar entrenadas, o sin entrenar.

## ResNet50V2

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
# from skimage.io import imread
import cv2

from tensorflow.keras.applications.resnet_v2 import ResNet50V2, decode_predictions, preprocess_input

In [5]:
# ResNet50V2??

In [2]:
# CODE MODEL
base_model = ResNet50V2(include_top= True, input_shape=(224, 224, 3), weights='imagenet', classifier_activation= 'softmax')


102869336/102869336 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [10]:
# base_model.summary()

In [3]:
c = base_model.layers[2]

In [11]:
c.strides

(2, 2)

In [12]:
# base_model.summary()

In [13]:
# CODE READ DATA

import os

def read_data(path):
    X = []
    for file in os.listdir(path):
        image = plt.imread(path + '/' + file)
        smallimage = cv2.resize(image, (224,224))
        print(path + '/' + file)

        X.append(smallimage)
    return np.array(X)


In [14]:

X_test = read_data('img')

#Preprocesar las imagenes tal y como entran en el model
X_test = preprocess_input(X_test)
print(X_test.shape)

img/batman.jpg
img/bear-1.jpg
img/cat.8016.jpg
img/cat.8037.jpg
img/dog.11856.jpg
img/dog.11857.jpg
img/horse.jpg
img/karate.jpg
img/pizza.jpg
(9, 224, 224, 3)


In [15]:
# CODE preds

preds = base_model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [16]:
len(preds)

9

In [17]:
len(preds[0])

1000

In [18]:
decodes = decode_predictions(preds, top=1)

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [19]:
decodes

[[('n02895154', 'breastplate', np.float32(0.6338938))],
 [('n02132136', 'brown_bear', np.float32(0.9994437))],
 [('n02124075', 'Egyptian_cat', np.float32(0.64985096))],
 [('n02124075', 'Egyptian_cat', np.float32(0.82098436))],
 [('n02106550', 'Rottweiler', np.float32(0.7329103))],
 [('n02106030', 'collie', np.float32(0.99525696))],
 [('n02113799', 'standard_poodle', np.float32(0.32455605))],
 [('n09835506', 'ballplayer', np.float32(0.5402341))],
 [('n07873807', 'pizza', np.float32(0.99713045))]]

In [20]:
# CODE decode preds
for j in decodes:
    print('####################')
    for i,decode in enumerate(j):
        print("Predicted:\n", decode[1], decode[2])

####################
Predicted:
 breastplate 0.6338938
####################
Predicted:
 brown_bear 0.9994437
####################
Predicted:
 Egyptian_cat 0.64985096
####################
Predicted:
 Egyptian_cat 0.82098436
####################
Predicted:
 Rottweiler 0.7329103
####################
Predicted:
 collie 0.99525696
####################
Predicted:
 standard_poodle 0.32455605
####################
Predicted:
 ballplayer 0.5402341
####################
Predicted:
 pizza 0.99713045


## VGG16
En este caso vamos a importar la red VGG16, que utilizaremos como red preentrenada y completaremos con una fully connected layer. 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

IM_SIZE=32

TRAIN_PATH = 'C:\\Users\\Miguel Angel\\Documents\\Bootcamp_DS_1_22\\datasets\\dogsandcats\\train'
filenames = os.listdir(TRAIN_PATH)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)
    
df = pd.DataFrame({
    'filenames': filenames,
    'category': categories
})

train_df, validate_df = train_test_split(df,
                                            test_size=0.20,
                                            random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [20]:
train_df.head()

,filenames,category
0,cat.3337.jpg,cat
1,dog.12039.jpg,dog
2,cat.2542.jpg,cat
3,dog.10283.jpg,dog
4,dog.10384.jpg,dog


In [ ]:
from keras.preprocessing.image import ImageDataGenerator


# Add our data-augmentation parameters to ImageDataGenerator data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 40,
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True)

# Note that the validation data should not be augmented!
validation_datagen = ImageDataGenerator(rescale = 1.0/255. )

In [ ]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                                TRAIN_PATH,
                                                                x_col='filenames',
                                                                y_col='category',
                                                                batch_size = 20,
                                                                class_mode = 'binary',
                                                                target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [24]:
from tensorflow.keras.applications.vgg16 import VGG16

# CODE model
base_model = VGG16(input_shape=(IM_SIZE,IM_SIZE,3),
                    include_top=False,
                    weights = 'imagenet'
                )

In [26]:
len(base_model.layers)

19

In [25]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [27]:

##### FULLY CONNECTED LAYER #####
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
    
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [29]:
len(model.layers)

23

In [28]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [29]:
vgghist = model.fit(train_generator,
                    validation_data = validation_generator,
                    batch_size = 128,
                    epochs = 10)

Epoch 1/10
263/263 [==============================] - 68s 257ms/step - loss: 0.6222 - acc: 0.6616 - val_loss: 0.5752 - val_acc: 0.7059
Epoch 2/10
263/263 [==============================] - 37s 140ms/step - loss: 0.5972 - acc: 0.6781 - val_loss: 0.5806 - val_acc: 0.7029
Epoch 3/10
263/263 [==============================] - 36s 136ms/step - loss: 0.5945 - acc: 0.6766 - val_loss: 0.5577 - val_acc: 0.7181
Epoch 4/10
263/263 [==============================] - 35s 134ms/step - loss: 0.5811 - acc: 0.6913 - val_loss: 0.5556 - val_acc: 0.7166
Epoch 5/10
263/263 [==============================] - 32s 123ms/step - loss: 0.5776 - acc: 0.6954 - val_loss: 0.5553 - val_acc: 0.7105
Epoch 6/10
263/263 [==============================] - 31s 118ms/step - loss: 0.5723 - acc: 0.6989 - val_loss: 0.5417 - val_acc: 0.7249
Epoch 7/10
263/263 [==============================] - 32s 121ms/step - loss: 0.5691 - acc: 0.7067 - val_loss: 0.5354 - val_acc: 0.7333
Epoch 8/10
263/263 [==============================] - 3

### Inception

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

IM_SIZE = 75

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                                TRAIN_PATH,
                                                                x_col='filenames',
                                                                y_col='category',
                                                                batch_size = 20,
                                                                class_mode = 'binary',
                                                                target_size = (IM_SIZE, IM_SIZE))



model.compile(optimizer = 'rmsprop',
                loss = 'binary_crossentropy',
                metrics = ['accuracy'])

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [32]:
model.fit(train_generator,
         validation_data = validation_generator,
         steps_per_epoch = 100,
         epochs = 5)

Epoch 1/5
100/100 [==============================] - 23s 186ms/step - loss: 0.9782 - accuracy: 0.6540 - val_loss: 0.5831 - val_accuracy: 0.7272
Epoch 2/5
100/100 [==============================] - 20s 205ms/step - loss: 0.6786 - accuracy: 0.6850 - val_loss: 0.6164 - val_accuracy: 0.7363
Epoch 3/5
100/100 [==============================] - 18s 177ms/step - loss: 0.6064 - accuracy: 0.6950 - val_loss: 0.5667 - val_accuracy: 0.7606
Epoch 4/5
100/100 [==============================] - 19s 189ms/step - loss: 0.6065 - accuracy: 0.7125 - val_loss: 0.6184 - val_accuracy: 0.6991
Epoch 5/5
100/100 [==============================] - 19s 186ms/step - loss: 0.5676 - accuracy: 0.7385 - val_loss: 0.5456 - val_accuracy: 0.7470


## ResNet50V2 sin entrenar

In [31]:
IM_SIZE=32

base_model = ResNet50V2(input_shape = (224, 224, 3),
                       include_top = False,
                       weights = None,
                       classifier_activation= "softmax")


train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    TRAIN_PATH,
                                                    x_col='filenames',
                                                    y_col='category',
                                                    batch_size = 20,
                                                    class_mode = 'binary',
                                                    target_size = (IM_SIZE, IM_SIZE))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_dataframe(validate_df,
                                                              TRAIN_PATH,
                                                              x_col='filenames',
                                                              y_col='category',
                                                              batch_size = 20,
                                                              class_mode = 'binary',
                                                              target_size = (IM_SIZE, IM_SIZE))

Found 5260 validated image filenames belonging to 2 classes.
Found 1316 validated image filenames belonging to 2 classes.


In [32]:
len(model.layers)

23

In [33]:
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)
    
model.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['acc'])

In [ ]:
model.fit(train_generator,
         validation_data = validation_generator,
         steps_per_epoch = 100,
         epochs = 5)

Epoch 1/5
100/100 [==============================] - 102s 975ms/step - loss: 3.5332 - acc: 0.8600 - val_loss: 0.4825 - val_acc: 0.9430
Epoch 2/5
100/100 [==============================] - 102s 1s/step - loss: 0.5437 - acc: 0.8930 - val_loss: 0.0976 - val_acc: 0.9605
